In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv


In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [5]:
df.columns = df.columns.str.lower().str.replace(" ","_")

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(" ","_")

In [6]:
df.head().T

,0,1,2,3,4
customerid,7590-vhveg,5575-gnvde,3668-qpybk,7795-cfocw,9237-hqitu
gender,female,male,male,male,female
seniorcitizen,0,0,0,0,0
partner,yes,no,no,no,no
dependents,no,no,no,no,no
tenure,1,34,2,45,2
phoneservice,no,yes,yes,no,yes
multiplelines,no_phone_service,no,no,no_phone_service,no
internetservice,dsl,dsl,dsl,dsl,fiber_optic
onlinesecurity,no,yes,yes,yes,no


In [7]:
df.dtypes
df.totalcharges=pd.to_numeric(df.totalcharges,errors='coerce')
df.totalcharges=df.totalcharges.fillna(0)
df.totalcharges.isnull().sum()

0

In [8]:
df.churn=(df.churn == 'yes').astype(int)
df.churn


0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: churn, Length: 7043, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split


In [10]:
df_full_train,df_test=train_test_split(df,test_size=0.2,random_state=1)


In [11]:
df_train,df_val=train_test_split(df_full_train,test_size=0.25,random_state=1)

In [12]:
len(df_train),len(df_test),len(df_val)

(4225, 1409, 1409)

In [13]:
df_train=df_train.reset_index(drop=True)

In [14]:
df_test=df_test.reset_index(drop=True)
df_val=df_val.reset_index(drop=True)

In [15]:
y_train=df_train.churn.values
y_test=df_test.churn.values
y_val=df_val.churn.values

In [16]:
y_TTest=df_test.churn.values

In [17]:
del df_train['churn']
del df_test['churn']
del df_val['churn']

**EDA**

In [18]:
df_full_train=df_full_train.reset_index(drop=True)

In [19]:
df_full_train.churn.value_counts(normalize=True)

churn
0    0.730032
1    0.269968
Name: proportion, dtype: float64

In [20]:
global_churn=df_full_train.churn.mean()
global_churn


0.26996805111821087

In [21]:
df_full_train.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [22]:
numeric=['tenure','monthlycharges', 'totalcharges']
categorical =['gender', 'seniorcitizen', 'partner', 'dependents'
              , 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod']

In [23]:
df_full_train[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

In [24]:
from IPython.display import display

In [25]:
for c in categorical:
        print(c)
        df_group=df_full_train.groupby(c).churn.agg(['mean','count'])
        df_group['diff']=df_group['mean']-global_churn
        df_group['risk']=df_group['mean']/global_churn
        display(df_group)
        print()
        print()
        

gender


,mean,count,diff,risk
gender,,,,
female,0.276824,2796,0.006856,1.025396
male,0.263214,2838,-0.006755,0.974980




seniorcitizen


,mean,count,diff,risk
seniorcitizen,,,,
0,0.242270,4722,-0.027698,0.897403
1,0.413377,912,0.143409,1.531208




partner


...

,mean,count,diff,risk
contract,,,,
month-to-month,0.431701,3104,0.161733,1.599082
one_year,0.120573,1186,-0.149395,0.446621
two_year,0.028274,1344,-0.241694,0.104730




paperlessbilling


,mean,count,diff,risk
paperlessbilling,,,,
no,0.172071,2313,-0.097897,0.637375
yes,0.338151,3321,0.068183,1.252560




paymentmethod


,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.168171,1219,-0.101797,0.622928
credit_card_(automatic),0.164339,1217,-0.105630,0.608733
electronic_check,0.455890,1893,0.185922,1.688682
mailed_check,0.193870,1305,-0.076098,0.718121


In [26]:
from sklearn.metrics import mutual_info_score

In [27]:
mutual_info_score(df_full_train.churn,df_full_train.gender)

0.0001174846211139946

In [28]:
def mutual_infoscore(series):
    return mutual_info_score(df_full_train.churn,series)

mi=df_full_train[categorical].apply(mutual_infoscore)

mi.sort_values(ascending=False)


contract            0.098320
onlinesecurity      0.063085
techsupport         0.061032
internetservice     0.055868
onlinebackup        0.046923
deviceprotection    0.043453
paymentmethod       0.043210
streamingtv         0.031853
streamingmovies     0.031581
paperlessbilling    0.017589
dependents          0.012346
partner             0.009968
seniorcitizen       0.009410
multiplelines       0.000857
phoneservice        0.000229
gender              0.000117
dtype: float64

**EDA CORRELATION** with numeric values

In [29]:
df_full_train[numeric].corrwith(df_full_train.churn)

tenure           -0.351885
monthlycharges    0.196805
totalcharges     -0.196353
dtype: float64

**ONE HOT ENCODING** (Encoding Categorical Variables)

In [30]:
from sklearn.feature_extraction import DictVectorizer

In [31]:
train_dicts=df_train[categorical+numeric].to_dict(orient='records')

In [32]:
dv=DictVectorizer(sparse=False)

In [33]:
x_train=dv.fit_transform(train_dicts)

**ENCODING THE VALIDATION SET**

In [34]:
val_dicts=df_val[categorical+numeric].to_dict(orient='records')
x_val=dv.transform(val_dicts)

**LOGISTIC REGRESSION**

In [35]:
from sklearn.linear_model import LogisticRegression

In [36]:
model=LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

In [37]:
y_predict_val=model.predict_proba(x_val)[:,1]

In [38]:
churn_decision=(y_predict_val>0.5)

In [39]:
(churn_decision == y_val).mean()

0.8034066713981547

In [40]:
dicts_full_train=df_full_train[categorical+numeric].to_dict(orient='records')

In [41]:
dv=DictVectorizer(sparse=False)

In [42]:
X_ft=dv.fit_transform(dicts_full_train)

In [43]:
y_ft=df_full_train.churn

In [44]:
model=LogisticRegression()

In [45]:
model.fit(X_ft,y_ft)

LogisticRegression()

In [46]:
dicts_test=df_test[categorical+numeric].to_dict(orient='records')
dv=DictVectorizer(sparse=False)
X_test=dv.fit_transform(dicts_test)

In [47]:
y_pred=model.predict_proba(X_test)

In [48]:
y_pred

array([[0.93775704, 0.06224296],
       [0.82526125, 0.17473875],
       [0.62973299, 0.37026701],
       ...,
       [0.99361995, 0.00638005],
       [0.83423903, 0.16576097],
       [0.40311479, 0.59688521]])

In [49]:
y_t=y_pred[:,1]

In [50]:
churn_d=(y_t >=0.5)

In [51]:
churn_d

array([False, False, False, ..., False, False,  True])

In [52]:
(y_TTest==churn_d).mean()

0.815471965933286